In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Base URL
base_url = "https://www.dubizzle.com.eg/en/properties/apartments-duplex-for-sale/?page={}"

# Initialize lists to store scraped data
apartments_links = []

# Pagination loop
page = 1
while True:
    print(f"Processing page {page}...")
    # Request the page
    url = base_url.format(page)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all product containers
    print("Finding apartments...")
    apartments = soup.find_all('div', class_='_637fa00f')

    if not apartments:
        print("No apartments found on this page. Stopping.")
        break  # Stop if no products found on the page

    print(f"Found {len(apartments)} apartments on this page.")
    for apartment in apartments:
        # Extract product link
        link = apartment.find('a', href=True)['href']
        full_link = f"https://www.dubizzle.com.eg{link}"  # Make sure the URL is complete
        apartments_links.append(full_link)

    # Increment page count
    page += 1

print("Scraping complete. Creating DataFrame...")
# Create DataFrame
df = pd.DataFrame({

    'Apartment Link': apartments_links
})

print("DataFrame created. Exporting to CSV...")
# Export to CSV
df.to_csv('dubizzle_apartments_links.csv', index=False)
print("CSV exported successfully!")

Streaming output truncated to the last 5000 lines.
Found 50 apartments on this page.
Processing page 3694...
Finding apartments...
Found 50 apartments on this page.
Processing page 3695...
Finding apartments...
Found 50 apartments on this page.
Processing page 3696...
Finding apartments...
Found 50 apartments on this page.
Processing page 3697...
Finding apartments...
Found 50 apartments on this page.
Processing page 3698...
Finding apartments...
Found 50 apartments on this page.
Processing page 3699...
Finding apartments...
Found 50 apartments on this page.
Processing page 3700...
Finding apartments...
Found 50 apartments on this page.
Processing page 3701...
Finding apartments...
Found 50 apartments on this page.
Processing page 3702...
Finding apartments...
Found 50 apartments on this page.
Processing page 3703...
Finding apartments...
Found 50 apartments on this page.
Processing page 3704...
Finding apartments...
Found 50 apartments on this page.
Processing page 3705...
Finding apa

KeyboardInterrupt: 

In [ ]:
# Create DataFrame
df = pd.DataFrame({

    'Apartment Link': apartments_links
})

# Export to CSV
df.to_csv('dubizzle_apartments_links.csv', index=False)

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

df=pd.read_csv('/content/dubizzle_apartments_links.csv')
apartments_links=df['Apartment Link'].tolist()


apartments_data = []


for link in tqdm(apartments_links, desc="Apartments Products", unit="Apartment"):
    response = requests.get(link)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Extract apartment details
    Highlights = soup.find_all('div', class_='_948d9e0a e655db77 _371e9918 _95d4067f')
    details = soup.find_all('div', class_='_9a8eacd9')
    location =  soup.find('span', {'aria-label': 'Location'})
    listing_date=soup.find('span', {'aria-label': 'Creation date'})
    price=soup.find('span', {'aria-label': 'Price'})
    down_payment=soup.find('span', {'aria-label': 'Price Info'})
    amenities=soup.find_all('span', class_='_241f82d9') + soup.find_all('span', class_='c327b807')

    apartment_data = {}

    for Highlight in Highlights:
        label = Highlight.find_all('span')[0].text.strip()
        value = Highlight.find_all('span')[1].text.strip()

        if label == 'Type':
            apartment_data['Type'] = value
        elif label == 'Ownership':
            apartment_data['Ownership'] = value
        elif label == 'Area (m²)':
            apartment_data['Area (m²)'] = value
        elif label == 'Bedrooms':
            apartment_data['Bedrooms'] = value
        elif label == 'Bathrooms':
            apartment_data['Bathrooms'] = value
        elif label == 'Furnished':
            apartment_data['Furnished'] = value


    for detail in details:
        label = detail.find_all('span')[0].text.strip()
        value = detail.find_all('span')[1].text.strip()

        if label == 'Payment Option':
            apartment_data['Payment Option'] = value
        elif label == 'Delivery Date':
            apartment_data['Delivery Date'] = value
        elif label == 'Level':
            apartment_data['Level'] = value
        elif label == 'Completion status':
            apartment_data['Completion status'] = value
        elif label == 'Delivery Term':
            apartment_data['Delivery Term'] = value

        apartment_data['Location'] = location.text.strip()
        apartment_data['Listing date'] = listing_date.text.strip()
        apartment_data['Price'] = price.text.strip()
        apartment_data['Down Payment'] = down_payment.text.strip() if down_payment else None
        apartment_data['Amenities'] = [amenity.text.strip() for amenity in amenities]

    apartments_data.append(apartment_data)


# Create DataFrame
    df = pd.DataFrame(apartments_data)


# Export to CSV
df.to_csv('dubizzle_apartments_data.csv', index=False)

Apartments Products:   0%|          | 498/267950 [05:19<47:39:30,  1.56Apartment/s]


KeyboardInterrupt: 

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

# Read the CSV file
df = pd.read_csv('/content/dubizzle_apartments_links.csv')
apartments_links = df['Apartment Link'].tolist()

# Function to extract data from a link
def extract_apartment_data(link):
    response = requests.get(link)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Extract apartment details
    Highlights = soup.find_all('div', class_='_948d9e0a e655db77 _371e9918 _95d4067f')
    details = soup.find_all('div', class_='_9a8eacd9')
    location = soup.find('span', {'aria-label': 'Location'})
    listing_date = soup.find('span', {'aria-label': 'Creation date'})
    price = soup.find('span', {'aria-label': 'Price'})
    down_payment = soup.find('span', {'aria-label': 'Price Info'})
    amenities = soup.find_all('span', class_='_241f82d9') + soup.find_all('span', class_='c327b807')

    apartment_data = {}

    for Highlight in Highlights:
        label = Highlight.find_all('span')[0].text.strip()
        value = Highlight.find_all('span')[1].text.strip()

        if label == 'Type':
            apartment_data['Type'] = value
        elif label == 'Ownership':
            apartment_data['Ownership'] = value
        elif label == 'Area (m²)':
            apartment_data['Area (m²)'] = value
        elif label == 'Bedrooms':
            apartment_data['Bedrooms'] = value
        elif label == 'Bathrooms':
            apartment_data['Bathrooms'] = value
        elif label == 'Furnished':
            apartment_data['Furnished'] = value

    for detail in details:
        label = detail.find_all('span')[0].text.strip()
        value = detail.find_all('span')[1].text.strip()

        if label == 'Payment Option':
            apartment_data['Payment Option'] = value
        elif label == 'Delivery Date':
            apartment_data['Delivery Date'] = value
        elif label == 'Level':
            apartment_data['Level'] = value
        elif label == 'Completion status':
            apartment_data['Completion status'] = value
        elif label == 'Delivery Term':
            apartment_data['Delivery Term'] = value

    apartment_data['Location'] = location.text.strip() if location else None
    apartment_data['Listing date'] = listing_date.text.strip() if listing_date else None
    apartment_data['Price'] = price.text.strip() if price else None
    apartment_data['Down Payment'] = down_payment.text.strip() if down_payment else None
    apartment_data['Amenities'] = [amenity.text.strip() for amenity in amenities]

    return apartment_data

# Create a list to store apartment data
apartments_data = []

# Use ThreadPoolExecutor to make requests in parallel
with ThreadPoolExecutor(max_workers=10) as executor:
    future_to_link = {executor.submit(extract_apartment_data, link): link for link in apartments_links}
    for future in tqdm(as_completed(future_to_link), total=len(apartments_links), desc="Apartments Products", unit="Apartment"):
        try:
            data = future.result()
            apartments_data.append(data)
        except Exception as e:
            print(f"Error processing link {future_to_link[future]}: {e}")

# Create DataFrame
df = pd.DataFrame(apartments_data)

# Export to CSV
df.to_csv('dubizzle_apartments_data.csv', index=False)

Apartments Products:   0%|          | 921/267950 [06:06<29:29:16,  2.52Apartment/s]


KeyboardInterrupt: 

In [ ]:
import aiohttp
import asyncio
from bs4 import BeautifulSoup
import pandas as pd
from tqdm.asyncio import tqdm as tqdm_asyncio
import time

# Read the CSV file
df = pd.read_csv('/content/dubizzle_apartments_links.csv')
apartments_links = df['Apartment Link'].tolist()

# Asynchronous function to extract data from a link
async def fetch(session, link):
    try:
        async with session.get(link) as response:
            if response.status == 200:
                response_text = await response.text()
                return response_text
            else:
                print(f"Failed to fetch {link}, Status Code: {response.status}")
                return None
    except Exception as e:
        print(f"Error fetching {link}: {e}")
        return None

# Function to parse apartment data from HTML content
def parse_apartment_data(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')

    # Extract apartment details
    Highlights = soup.find_all('div', class_='_948d9e0a e655db77 _371e9918 _95d4067f')
    details = soup.find_all('div', class_='_9a8eacd9')
    location = soup.find('span', {'aria-label': 'Location'})
    listing_date = soup.find('span', {'aria-label': 'Creation date'})
    price = soup.find('span', {'aria-label': 'Price'})
    down_payment = soup.find('span', {'aria-label': 'Price Info'})
    amenities = soup.find_all('span', class_='_241f82d9') + soup.find_all('span', class_='c327b807')

    apartment_data = {}

    for Highlight in Highlights:
        spans = Highlight.find_all('span')
        if len(spans) >= 2:
            label = spans[0].text.strip()
            value = spans[1].text.strip()
            apartment_data[label] = value

    for detail in details:
        spans = detail.find_all('span')
        if len(spans) >= 2:
            label = spans[0].text.strip()
            value = spans[1].text.strip()
            apartment_data[label] = value

    apartment_data['Location'] = location.text.strip() if location else None
    apartment_data['Listing date'] = listing_date.text.strip() if listing_date else None
    apartment_data['Price'] = price.text.strip() if price else None
    apartment_data['Down Payment'] = down_payment.text.strip() if down_payment else None
    apartment_data['Amenities'] = [amenity.text.strip() for amenity in amenities]

    return apartment_data

# Function to handle scraping with rate limiting and batch processing
async def scrape_apartment_data(links, batch_size=1000, max_workers=50):
    apartments_data = []

    # Process links in batches
    for i in range(0, len(links), batch_size):
        batch_links = links[i:i + batch_size]
        print(f"Processing batch {i // batch_size + 1} / {len(links) // batch_size + 1}")

        async with aiohttp.ClientSession() as session:
            tasks = []
            for link in batch_links:
                tasks.append(fetch(session, link))

            # Gather the responses asynchronously with a limit on concurrency
            for task in tqdm_asyncio.as_completed(tasks, total=len(tasks), desc="Apartments Products", unit="Apartment"):
                try:
                    html_content = await task
                    if html_content:
                        data = parse_apartment_data(html_content)
                        apartments_data.append(data)
                except Exception as e:
                    print(f"Error processing task: {e}")

        # Save progress after each batch to prevent data loss
        df = pd.DataFrame(apartments_data)
        df.to_csv(f'dubizzle_apartments_data_batch_{i // batch_size + 1}.csv', index=False)
        print(f"Batch {i // batch_size + 1} saved!")

        # Sleep for a few seconds to prevent overwhelming the server
        await asyncio.sleep(5)

    return apartments_data

# Run the scraping and save the data to a CSV file
async def main():
    # Set batch size and max concurrent workers (tune these values based on your system and server load)
    batch_size = 1000  # Number of links processed in each batch
    max_workers = 50   # Max concurrent requests at a time

    apartments_data = await scrape_apartment_data(apartments_links, batch_size=batch_size, max_workers=max_workers)

    # Create final DataFrame and export to CSV
    df = pd.DataFrame(apartments_data)
    df.to_csv('dubizzle_apartments_data_full.csv', index=False)

# In Google Colab, use await directly to execute the main function
await main()


Processing batch 1 / 268


Apartments Products:   0%|          | 1/1000 [00:01<17:02,  1.02s/Apartment]

Failed to fetch https://www.dubizzle.com.eg/en/ad/%D8%B4%D9%82%D8%A9-%D9%84%D9%84%D8%A8%D9%8A%D8%B9176%D9%85-%D8%A8%D9%83%D9%85%D8%A8%D9%88%D9%86%D8%AF-%D8%A7%D9%84%D9%85%D8%B9%D8%A7%D8%AF%D9%8A-%D8%AC%D8%A7%D8%B1%D8%AF%D9%86%D8%B2-%D8%A7%D9%88%D9%84-%D8%B3%D9%83%D9%86-ID202489797.html, Status Code: 404


Apartments Products:  14%|█▍        | 141/1000 [00:20<01:32,  9.29Apartment/s]

Failed to fetch https://www.dubizzle.com.eg/en/ad/%D8%B4%D9%82%D8%A9-%D9%84%D9%84%D8%A8%D9%8A%D8%B9-%D8%A8%D9%85%D8%AF%D9%8A%D9%86%D8%A9-%D8%A7%D9%84%D8%B4%D8%B1%D9%88%D9%82-%D9%85%D9%86%D8%B7%D9%82%D8%A9-%D8%A7%D9%8470-%D9%85%D8%AA%D8%B1-ID202509689.html, Status Code: 404


...

Apartments Products:   4%|▎         | 36/1000 [00:06<01:38,  9.77Apartment/s]

Failed to fetch https://www.dubizzle.com.eg/en/ad/i-villa-roof-240m-direct-lagoon-for-sale-in-mountain-view-i-city-new-cairo-%D9%85%D8%A7%D9%88%D9%86%D8%AA%D9%86-%D9%81%D9%8A%D9%88-%D8%A7%D9%8A-%D8%B3%D9%8A%D8%AA%D9%8A-%D8%A7%D9%84%D8%AA%D8%AC%D9%85%D8%B9-%D8%A7%D9%84%D8%AE%D8%A7%D9%85%D8%B3-ID500984404.html, Status Code: 404


Apartments Products:   5%|▌         | 51/1000 [00:10<03:09,  5.00Apartment/s]


CancelledError: 

Error fetching https://www.dubizzle.com.eg/en/ad/resale-apartment-m-v-icity-october-ready-to-move-ID500970264.html: 'NoneType' object has no attribute 'connect'
Error fetching https://www.dubizzle.com.eg/en/ad/direct-from-owner-mountian-view-icity-oct-ready-to-move-200m-aprt-ID202422747.html: 'NoneType' object has no attribute 'connect'
Error fetching https://www.dubizzle.com.eg/en/ad/apartment-for-sale-with-catchy-price-and-very-prime-location-in-lake-view-residence-2-ID500971693.html: 'NoneType' object has no attribute 'connect'
Error fetching https://www.dubizzle.com.eg/en/ad/%D9%81%D8%B1%D8%B5%D8%A9-%D9%84%D8%A7-%D8%AA%D9%81%D9%88%D8%AA-%D8%B4%D9%82%D9%87-%D9%84%D9%84%D8%A8%D9%8A%D8%B9-160-%D9%85%D8%AA%D8%B1-%D9%81%D9%8A-%D8%A7%D9%85%D9%8A%D8%B2-%D9%85%D9%88%D9%82%D8%B9-%D8%A7%D9%84%D8%B1%D8%AD%D8%A7%D8%A8-rehab-city-ID202482554.html: 'NoneType' object has no attribute 'connect'
Error fetching https://www.dubizzle.com.eg/en/ad/%D8%B3%D8%AA%D8%AF%D9%8A%D9%88-%D9%84%D9%84%D8%A8%D9%8A